In [1]:
import os
from pathlib import Path
import glob
import numpy as np
import imageio
import zarr

# Inputs
pred_mask_dir = "/mnt/efs/aimbl_2025/student_data/S-LS/stardist/ON_stardist_masks_tif"
raw_dir  = "/mnt/efs/aimbl_2025/student_data/S-LS/raw_bacteria"
marker_dir = "/mnt/efs/aimbl_2025/student_data/S-LS/tif_marker"


# Output Zarr store
store_path = "/mnt/efs/aimbl_2025/student_data/S-LS/marker_data.zarr"
zarr.open(store_path, zarr_version=2)

# Open/create root and subgroups
root = zarr.group(store_path)
pred_masks_grp = root.create_group("pred_mask_bacteria") 
raw_grp   = root.create_group("bacteria")    
marker_grp = root.create_group("marker")    

def stack_and_write(img_folder, group, dataset_name, chunks=(1,552,688)):
    files = sorted(glob.glob(os.path.join(img_folder, "*.tif")))
    N = len(files)
    # Load all images, stack to (N, H, W)
    stack = np.stack([imageio.imread(f) for f in files], axis=0)
    N, H, W = stack.shape
    

    ds = group.create_array(name=dataset_name, shape=(N, H, W), chunks=chunks, dtype="uint16")
    ds[...] = stack

    ds.attrs["filenames"] = [Path(f).name for f in files]

    print(f" Wrote '{group.path}/{dataset_name}' | shape={(N,H,W)} | chunks={chunks} | dtype=uint16")

# Build the two stacks
stack_and_write(pred_mask_dir, pred_masks_grp, "pred_mask_stack", chunks=(1,552,688))
stack_and_write(raw_dir,  raw_grp,   "raw_bacteria_stack",   chunks=(1,552,688))
stack_and_write(marker_dir,  marker_grp,   "marker_stack",   chunks=(1,552,688))


print(root.tree())
print("hola")


/tmp/ipykernel_733603/3807115584.py:28: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  stack = np.stack([imageio.imread(f) for f in files], axis=0)


 Wrote 'pred_mask_bacteria/pred_mask_stack' | shape=(72, 2208, 2752) | chunks=(1, 552, 688) | dtype=uint16
 Wrote 'bacteria/raw_bacteria_stack' | shape=(72, 2208, 2752) | chunks=(1, 552, 688) | dtype=uint16
 Wrote 'marker/marker_stack' | shape=(72, 2208, 2752) | chunks=(1, 552, 688) | dtype=uint16


/
├── bacteria
│   └── raw_bacteria_stack (72, 2208, 2752) uint16
├── marker
│   └── marker_stack (72, 2208, 2752) uint16
└── pred_mask_bacteria
    └── pred_mask_stack (72, 2208, 2752) uint16


hola
